In [ ]:
! pip install deap

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

# Problema da Mochila (Exemplo dado)

In [ ]:
# Definição do problema multiobjetivo (minimizar peso, maximizar valor)
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))  # Minimiza peso, maximiza valor
creator.create("Individual", list, fitness=creator.FitnessMulti)

In [ ]:
# Instância para teste
# Definição dos itens da mochila (peso, valor)
ITENS = [
    (3, 10), (8, 30), (2, 12), (6, 28), (5, 18),
    (12, 50), (7, 20), (4, 15), (9, 38), (10, 35),
    (15, 60), (1, 5), (11, 40), (14, 48), (13, 45),
    (16, 55), (17, 62), (18, 65), (19, 70), (20, 75),
    (5, 10), (7, 15), (6, 12), (9, 25), (12, 42),
    (14, 45), (10, 18), (3, 8), (8, 22), (4, 14)
]

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
# Função de avaliação
def evaluate(individual):
    peso_total = sum(ITENS[i][0] for i in range(len(ITENS)) if individual[i] == 1)
    valor_total = sum(ITENS[i][1] for i in range(len(ITENS)) if individual[i] == 1)

    return peso_total, valor_total

In [ ]:
# Configuração da população
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(ITENS))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", evaluate)

In [ ]:
# Parâmetros do algoritmo
NGEN = 100  # Número de gerações
POP_SIZE = 100  # Tamanho da população
CXPB, MUTPB = 0.7, 0.2  # Probabilidades de cruzamento e mutação

# Inicialização
def main():
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
                                         stats=stats, halloffame=hof, verbose=True)

    # Coletar dados para o gráfico
    pesos = [ind.fitness.values[0] for ind in pop]
    valores = [ind.fitness.values[1] for ind in pop]
    pareto_pesos = [ind.fitness.values[0] for ind in hof]
    pareto_valores = [ind.fitness.values[1] for ind in hof]

    # Identificar soluções dominadas
    dominadas = [(pesos[i], valores[i]) for i in range(len(pesos)) if (pesos[i], valores[i]) not in zip(pareto_pesos, pareto_valores)]
    if dominadas:
        dominadas_pesos, dominadas_valores = zip(*dominadas)
    else:
        dominadas_pesos, dominadas_valores = [], []

    # Plotando a fronteira de Pareto e soluções dominadas
    plt.scatter(dominadas_pesos, dominadas_valores, c='gray', alpha=0.5, label="Soluções Dominadas")
    plt.scatter(pareto_pesos, pareto_valores, c='red', label="Fronteira de Pareto")
    plt.xlabel("Peso Total")
    plt.ylabel("Valor Total")
    plt.title("Fronteira de Pareto: Peso vs Valor")
    plt.legend()
    plt.grid()
    plt.show()

    return pop, log, hof

if __name__ == "__main__":
    main()

# Exercício Prático - Ações

In [ ]:
# @title Definição do problema multiobjetivo (Maximizar Retorno, Minimizar Risco)
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Maximiza retorno, minimiza risco
creator.create("Individual", list, fitness=creator.FitnessMulti)

In [ ]:
# @title Instância de teste
# formato: (Retorno, risco)
ACOES = [
  (12, 8), (15, 10), (10, 6), (18, 12), (14, 9),
  (8, 5), (16, 11), (11, 7), (13, 8), (9, 6)
]

Q_ACOES = len(ACOES)

In [ ]:
# @title Função de avaliação
def evaluate(individual):

    ind = np.array(individual)
    ind[ind < 0] = 0 # Não há peso negativo

    # Normalizando
    ind = (ind - np.min(ind)) / (np.max(ind) - np.min(ind))

    # Calculo com base no vetor de pesos
    retorno_total = sum(ACOES[i][0] * ind[i] for i in range(Q_ACOES))
    risco_total = sum(ACOES[i][1] * ind[i] for i in range(Q_ACOES))

    return retorno_total, risco_total

In [ ]:
# @title Configuração da toolbox
toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=Q_ACOES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", evaluate)

In [ ]:
# @title Pipeline experimental

# Parâmetros do algoritmo
NGEN = 100  # Número de gerações
POP_SIZE = 100  # Tamanho da população
CXPB, MUTPB = 0.7, 0.2  # Probabilidades de cruzamento e mutação

# Inicialização
def main():
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
                                         stats=stats, halloffame=hof, verbose=True)

    # Coletar dados para o gráfico
    # formato: (Retorno, risco)
    retornos = [ind.fitness.values[0] for ind in pop]
    riscos = [ind.fitness.values[1] for ind in pop]
    pareto_retornos = [ind.fitness.values[0] for ind in hof]
    pareto_riscos = [ind.fitness.values[1] for ind in hof]

    # Identificar soluções dominadas
    dominadas = [(riscos[i], retornos[i]) for i in range(len(riscos)) if (riscos[i], retornos[i]) not in zip(pareto_riscos, pareto_retornos)]
    if dominadas:
        dominadas_riscos, dominadas_retornos = zip(*dominadas)
    else:
        dominadas_riscos, dominadas_retornos = [], []

    # Plotando a fronteira de Pareto e soluções dominadas
    plt.scatter(dominadas_riscos, dominadas_retornos, c='gray', alpha=0.5, label="Soluções Dominadas")
    plt.scatter(pareto_riscos, pareto_retornos, c='red', label="Fronteira de Pareto")
    plt.xlabel("Risco Total")
    plt.ylabel("Retorno Total")
    plt.title("Fronteira de Pareto: Risco vs Retorno")
    plt.legend()
    plt.grid()
    plt.show()

    return pop, log, hof

if __name__ == "__main__":
    main()